<a href="https://colab.research.google.com/github/bees4ever/ai-workshop-2022/blob/main/YOLO_Workshop_Capgemini_2022_Exercise_00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crime Detection Workshop
*Capgemini Challenge*

Catch the thief through image analysis and prove your analytical skill!

Estimated Duration: 10 min

Challange Points: 0 

## Exercise 0: Click & Learn
In the following notebook we provide several excersices to detect a suspicious person of images of a suveillance camera.

This Exercise is about to get ready with YOLO. Follow the prepared instructions to import the yolo framework and also some sample data set to do a first object detections.

Also, this notebook helps you to solve all other task. Try to understand all the different data processing and model forecasting steps, which you can use later.






Note: The data are downloaded from kaggle and underly different lizenses, even unkown.

- https://www.kaggle.com/datasets/odins0n/ucf-crime-dataset (CC0 lizense)
- https://www.kaggle.com/datasets/fmena14/crowd-counting (unkown)



## Download YOLO
Download the YOLO framework using the prepared dataset.

In [ ]:
!wget https://raw.githubusercontent.com/bees4ever/ai-workshop-2022/main/yolo_sample/requirements.txt
!pip install -r requirements.txt
%matplotlib inline
!pip install PIL

## Download Sample Dataset

Download some sample Image

In [ ]:
!wget https://github.com/bees4ever/ai-workshop-2022/raw/main/yolo_sample/seq_000018.jpg

## Get ready with YOLO

Your task is to read the `README` of `https://github.com/ultralytics/yolov5` and get ready with YOLO:

- find a matching YOLO model on Torch
- analyse the downloaded data from previous step
- which kind of data you can retreive from prediction
- show some bounding boxes of detected objects

In [ ]:
# There are different possibilities to work with the YOLO model. 
# In this example we load the model from torch hub
import torch
# The variable `model` contains the yolov3 model.
# This means, we can use `model` and insert a path to an image. 
# The return value contains the complete YOLO forecast
model = torch.hub.load('ultralytics/yolov3', 'yolov3')



In [ ]:
#' Let's have an example:
#' We loaded the file 'seq_000018.jpg', it is availbe here on colab
img = 'seq_000018.jpg'
# Passing the img path to `model`, returns an results object
results = model(img)

# Let's spent some time to analyse `results`.



In [ ]:
# We see, results is an object of class models.common.Detections. Searching
# for pytorch objects could help here
print(results)
print(type(results))

In [ ]:
# We will just play a bit around and investigate the possibilities
# with results.pred, we get the prediction made from YOLO. The prediction 
# is done for every sequence provided, which in this case is only one
# sequence
results.pred


In [ ]:
# With `results` we can save all detected objects to a folder, this can be done
# using:

crops = results.crop(save=True)

# In runs/detect/exp/crops you see all detected objects listed.

# Using results.crop(save=False) the output is not saved to disk,
# BUT in both cases in variable `crops` we have all found objects gathered 
# in one array, let's see:

print(crops[0])

In [ ]:
#' mapping the crops to the label we get a simple overview of available items
# of this specific image
list(map(lambda x: x['label'], crops))

In [ ]:
#' A similiar result can be archived using 
results.pandas().xywh[0].name

In [ ]:
#' Here we get the clear name as a pandas Dataframe, which can be re used for 
# further calculations like object counting:
amount_handbags = sum("handbag" == results.pandas().xywh[0].name)
print(f"We found {amount_handbags} handbags")

In [ ]:
# by the way, pandas Dataframes are helpfull to glue data together.
# Let's say we want to collect the amount of handbags to the image number. 
from pandas import DataFrame
# Create the Dataframe based on a `dict`
df = DataFrame({'handbags': [0, 5, 2, 12, 3, 4], 'image': ['seq1', 'seq2', 'seq3', 'seq4', 'seq5', 'seq6']})

# Then you can plot it, using:
%matplotlib inline
df.plot('image', 'handbags')


In [ ]:
#' Or you can create a histogram of available objects
df = results.pandas().xywh[0].name
df.hist()

In [ ]:
# Print the results.pandas().xywh[0] to see the full inside of the prediction in DataFrame format
results.pandas().xywh[0]

### Using the extracted crops

In [ ]:
# Some steps before, we saw how to extract the crops.
# There are different possibilities to work with the extracted objects.
# For this reason it is usefull to take some time and think about
# how this crops can be easily loaded into python.

# Use the os library
import os

# For nice progressbars it is always cool to have it enabled
import progressbar as pb
import matplotlib

# Load image processing tool of python
import cv2

# define the main dir
main_dir = 'runs/detect/exp/crops/handbag'



# We want to save all width and heights of the handbags in a dataframe
df = {'width': [], 'height': [], 'imgname': []}
for img in pb.progressbar(os.listdir(main_dir)):
    img_path = os.path.join(main_dir, img) 
    shape = cv2.imread(img_path).shape
    # shape is `shape == (480, 640, 3)` the height, width and dimension of the image
    df['height'].append(shape[0])
    df['width'].append(shape[1])
    df['imgname'].append(img_path)
    
    
df = DataFrame(df)
df


In [ ]:
#' It also can help to create a histogram of the image:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from google.colab.patches import cv2_imshow
import os
from pandas import DataFrame

color = ('b','g','r')
img_parsed = cv2.imread('seq_000018.jpg', -1)
for channel,col in enumerate(color):
    histr = cv2.calcHist([img_parsed], [channel], None, [256], [0,256])
    plt.plot(histr, color = col)
    plt.xlim([0, 256])
plt.title('Histogram for color scale picture of seq_000018.jpg')
plt.show()

In [ ]:
#' Let create a function which creates the analysis of this three channels

"""
:param: img_path the path to the image source
:return: the color deviation of all three channels red, green, blue
"""
def img_hist(img_path):
  color = ('b','g','r')
  res = dict()
  img_parsed = cv2.imread('seq_000018.jpg', -1)
  for channel,col in enumerate(color):
    histr = cv2.calcHist([img_parsed], [channel], None, [256], [0,256])
    res[col] = histr.reshape(256)
  
  return res

df = img_hist('seq_000018.jpg')
print(df.keys())
DataFrame(df)


In [ ]:
# The color deviation of the channel can give a usefull insight of a image frame
# If the color channels are the same, then the image could be the same

## Optional: Work with a different YOLO model

Now it is your turn - load different models from pythorch and re-run the above lines




In [ ]:
# Your turn here

### Work with the YOLO Tools

The YOLO Github repo contains different tools / scripts which helps to predict different image sources, also videos and made it possible to fine tune the model.

First, we download the repo. Then we go through the detection and training toolset.

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
# Use the prediction tools from YOLO
!python yolov5/detect.py --source /content/seq_000018.jpg  --save-crop
